<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/aprendizaje_reforzado/q_aprendizaje_taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ejemplo de diseño: Taxi auto-conducido

En este ejemplo se va a diseñar una simulación de un taxi auto-conducido. El objetivo principal es demostrar, en un entorno simplificado, cómo se pueden utilizar las técnicas de RL para desarrollar un enfoque eficiente y seguro para abordar el problema.

El trabajo del SmartCab es recoger al pasajero en un lugar y dejarlo en otro. Estas son algunas de las cosas de las que nos gustaría que nuestro smartcab se encargara:

- Dejar al pasajero en el lugar correcto
- Ahorrar tiempo al pasajero tardando lo mínimo posible en dejarle.
- Cuidar la seguridad del pasajero y las normas de tráfico

Hay diferentes aspectos que hay que considerar aquí mientras se modela una solución RL para este problema: recompensas, estados y acciones.

#### 1 Recompensas

Dado que el agente (el conductor imaginario) está motivado por la recompensa y va a aprender a controlar el taxi mediante experiencias de prueba de entorno, tenemos que decidir las recompensas y/o penalizaciones y su magnitud en consecuencia. He aquí algunos puntos a tener en cuenta:

- El agente debe recibir una alta recompensa positiva por una entrega exitosa porque este comportamiento es muy deseado
- El agente debe ser penalizado si intenta dejar a un pasajero en lugares equivocados
- El agente debería recibir una ligera recompensa negativa por no llegar al destino después de cada paso de tiempo. "Ligeramente" negativo porque preferimos que nuestro agente llegue tarde en lugar de hacer movimientos erróneos tratando de llegar al destino lo más rápido posible.

#### 2. Espacio de estados

En el aprendizaje por refuerzo, el agente se encuentra con un estado, y entonces toma una acción de acuerdo con el estado en el que se encuentra.

El espacio de estados es el conjunto de todas las situaciones posibles en las que puede encontrarse nuestro taxi. El estado debe contener información útil que el agente necesita para realizar la acción correcta.

Digamos que tenemos un área de entrenamiento para nuestro SmartCab en el que le estamos enseñando a transportar personas en un aparcamiento a cuatro lugares diferentes (R,G,Y,B):

<img src='https://storage.googleapis.com/lds-media/images/Reinforcement_Learning_Taxi_Env.width-1200.png'>

Supongamos que Smartcab es el único vehículo en este estacionamiento. Podemos dividir el estacionamiento en una cuadrícula de 5x5, lo que nos brinda 25 posibles ubicaciones de taxis. Estos 25 lugares son una parte de nuestro espacio de estados. Observe que el estado de ubicación actual de nuestro taxi es la coordenada (3, 1).

También notará que hay cuatro (4) ubicaciones en las que podemos recoger y dejar a un pasajero: R, G, Y, B o `[(0,0), (0,4), (4,0), (4,3)]` en (fila, columna) coordenadas. Nuestro pasajero ilustrado está en la ubicación Y y desea ir a la ubicación R.

Cuando también contabilizamos un (1) estado de pasajero adicional dentro del taxi, podemos tomar todas las combinaciones de ubicaciones de pasajeros y ubicaciones de destino para llegar a un número total de estados para nuestro entorno de taxi; hay cuatro (4) destinos y cinco (4 + 1) ubicaciones de pasajeros.

Por lo tanto, nuestro entorno de taxi tiene un total de 5x5x5x4=500 estados posibles.

#### 3. Espacio de acción

El agente se encuentra con uno de los 500 estados y realiza una acción. La acción en nuestro caso puede ser moverse en una dirección o decidir recoger/dejar a un pasajero.

En otras palabras, tenemos seis acciones posibles:

1. Sur
2. Norte
3. Este
4. Oeste
5. Recoger un pasajero
6. Dejar al pasajero


Este es el espacio de acción : el conjunto de todas las acciones que nuestro agente puede realizar en un estado dado.

Notarás en la ilustración de arriba que el taxi no puede realizar ciertas acciones en ciertos estados debido a las paredes. En el código del entorno, simplemente proporcionaremos una penalización de -1 por cada pared golpeada y el taxi no se moverá a ningún lado. Esto solo acumulará multas que harán que el taxi considere dar la vuelta a la pared.

### Implementación con Python

Afortunadamente, OpenAI Gym tiene este entorno exacto ya creado para nosotros.

Gym proporciona diferentes entornos de juego que podemos conectar a nuestro código y probar un agente. La biblioteca se encarga de la API para proporcionar toda la información que nuestro agente requiera, como posibles acciones, puntuación y estado actual. Solo tenemos que centrarnos en la parte del algoritmo para nuestro agente.

Usaremos el entorno Gym llamado Taxi-V2, del cual se extrajeron todos los detalles explicados anteriormente. Los objetivos, recompensas y acciones son todos iguales.

Necesitamos instalar gymprimero. Ejecutar lo siguiente en un cuaderno Jupyter debería funcionar:

In [1]:
# Para ejecuciones fuera del entorno de Google Colab, ejecutar:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install gym[toy_text]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 63.0 MB/s eta 0:00:00


Una vez instalado, podemos cargar el entorno del juego y renderizar lo que parece:

In [12]:
import gym
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [18]:
env = gym.make("Taxi-v3").env
disable_render_order_enforcing=True
env.reset()
env.render()

In [16]:
import gym
import time
from IPython.display import clear_output

env = gym.make("Taxi-v3").env

# for i in range(0,100):
#     clear_output(wait=True)
#     env.reset()
#     env.render()
#     time.sleep(0.5)

In [19]:
env.s = 328
env.render()
print(env.step(2))
time.sleep(10)
clear_output(wait=True)
env.render()

(407, -1, False, {'prob': 1.0, 'action_mask': array([0, 1, 0, 0, 0, 0], dtype=int8)})


https://www.kaggle.com/code/karthikcs1/reinforcement-learning-taxi-v3-openai/notebook

La interfaz núcleo de la librería gym es `env`, que es la interfaz de entorno unificado. Los siguientes son los métodos `env` que nos serían muy útiles:

- `env.reset`: reinicia el entorno y devuelve un estado inicial aleatorio.
- `env.step(action)`: Paso el medio ambiente por un paso de tiempo. Devoluciones

* observation : Observaciones del entorno
* reward : si tu acción fue beneficiosa o no
* done : Indica si hemos recogido y dejado con éxito a un pasajero, también llamado episodio único
* info : información adicional como el rendimiento y la latencia con fines de depuración

- `env.render`: Representa un cuadro del entorno (útil para visualizar el entorno)


Nota: estamos usando al `.env` final de `make` para evitar que el entrenamiento se detenga en 200 iteraciones, que es el valor predeterminado para la nueva versión de Gym ( referencia ).

## Recordatorio de nuestro problema

Aquí está nuestra declaración de problema reestructurada (de Gym docs):

"Hay 4 ubicaciones (etiquetadas con letras diferentes), y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Recibimos +20 puntos por una entrega exitosa y perdemos 1 punto por cada vez- paso que toma. También hay una penalización de 10 puntos por acciones ilegales de recogida y entrega".

Vamos a sumergirnos más en el medio ambiente.

In [14]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


- El cuadrado relleno representa el taxi, que es amarillo sin pasajero y verde con pasajero.
- La tubería ("|") representa una pared que el taxi no puede cruzar.
- R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa el lugar actual de recogida de pasajeros y la letra morada es el destino actual.

Como se verifica en las impresiones, tenemos un espacio de acción de tamaño 6 y un espacio de estado de tamaño 500. Como verá, nuestro algoritmo RL no necesitará más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de manera única asignando un número único a cada estado posible, y RL aprende a elegir un número de acción del 0 al 5 donde:

- 0 = sur
- 1 = norte
- 2 = este
- 3 = oeste
- 4 = recoger
- 5 = dejar

Recuerde que los 500 estados corresponden a una codificación de la ubicación del taxi, la ubicación del pasajero y la ubicación de destino.

El aprendizaje por refuerzo aprenderá un mapeo de estados a la acción óptima para realizar en ese estado por exploración , es decir, el agente explora el entorno y realiza acciones en función de las recompensas definidas en el entorno.

La acción óptima para cada estado es la acción que tiene la mayor recompensa acumulada a largo plazo.

## Volver a nuestra ilustración

De hecho, podemos tomar nuestra ilustración anterior, codificar su estado y dárselo al entorno para que se renderice en Gym. Recuerde que tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino es la ubicación 0. Usando el método de codificación de estado Taxi-v2, podemos hacer lo siguiente:

In [15]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328


Estamos usando las coordenadas de nuestra ilustración para generar un número correspondiente a un estado entre 0 y 499, que resulta ser 328 para el estado de nuestra ilustración.

Luego, podemos configurar el estado del entorno manualmente `env.env.s`usando ese número codificado. Puedes jugar con los números y verás que el taxi, el pasajero y el destino se mueven.